In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import pickle

from plot_3d_helpers import expanded_detector_boundary_points, detector_boundary_points
from plot_3d_helpers import fps_clustering_downsample, get_min_dists, energy_weighted_density_sampling


# Loading File

In [2]:
f = uproot.open("input_files/bdt_convert_superunified_bnb_ncpi0_full_spacepoints.root")


In [3]:
rse = f["wcpselection"]["T_eval"].arrays(["run", "subrun", "event"], library="np")

true_nu_vtx = f["wcpselection"]["T_eval"].arrays(["truth_vtxX", "truth_vtxY", "truth_vtxZ"], library="np")
true_nu_vtx = np.stack([true_nu_vtx["truth_vtxX"], true_nu_vtx["truth_vtxY"], true_nu_vtx["truth_vtxZ"]], axis=-1)

reco_nu_vtx = f["wcpselection"]["T_PFeval"].arrays(["reco_nuvtxX", "reco_nuvtxY", "reco_nuvtxZ"], library="np")
reco_nu_vtx = np.stack([reco_nu_vtx["reco_nuvtxX"], reco_nu_vtx["reco_nuvtxY"], reco_nu_vtx["reco_nuvtxZ"]], axis=-1)


In [4]:
# re-formatting the spacepoints before downsampling

num_events = 70
# num_events = len(truth_vtx["truth_vtxX"])

spacepoints = f["wcpselection"]["T_spacepoints"].arrays(["Tcluster_spacepoints_x", 
                                                                 "Tcluster_spacepoints_y", 
                                                                 "Tcluster_spacepoints_z", 
                                                                 #"Tcluster_spacepoints_q",
                                                                 "Trec_spacepoints_x", 
                                                                 "Trec_spacepoints_y", 
                                                                 "Trec_spacepoints_z", 
                                                                 #"Trec_spacepoints_q",
                                                                 "TrueEDep_spacepoints_startx",
                                                                 "TrueEDep_spacepoints_starty",
                                                                 "TrueEDep_spacepoints_startz",
                                                                 "TrueEDep_spacepoints_endx",
                                                                 "TrueEDep_spacepoints_endy",
                                                                 "TrueEDep_spacepoints_endz",
                                                                 "TrueEDep_spacepoints_edep",
                                                                 ], 
                                                                 entry_start=0, entry_stop=num_events, library="np")

true_pi0_spacepoints = f["nuselection"]["NeutrinoSelectionFilter"].arrays(["pi0truth_gamma1_xpos",
                                                                           "pi0truth_gamma1_ypos",
                                                                           "pi0truth_gamma1_zpos",
                                                                           "pi0truth_gamma1_edep",
                                                                           "pi0truth_gamma2_xpos",
                                                                           "pi0truth_gamma2_ypos",
                                                                           "pi0truth_gamma2_zpos",
                                                                           "pi0truth_gamma2_edep",
                                                                           ], 
                                                                           entry_start=0, entry_stop=num_events, library="np")

Tcluster_spacepoints = []
Trec_spacepoints = []
TrueEDep_spacepoints = []
TrueEDep_spacepoints_edep = []
true_gamma1_EDep_spacepoints = []
true_gamma1_EDep_spacepoints_edep = []
true_gamma2_EDep_spacepoints = []
true_gamma2_EDep_spacepoints_edep = []
for event_i in range(num_events):

    # reconstructed spacepoints 
    Tcluster_spacepoints.append(np.stack([spacepoints["Tcluster_spacepoints_x"][event_i],
                                         spacepoints["Tcluster_spacepoints_y"][event_i],
                                         spacepoints["Tcluster_spacepoints_z"][event_i]], axis=-1))
    Trec_spacepoints.append(np.stack([spacepoints["Trec_spacepoints_x"][event_i],
                                      spacepoints["Trec_spacepoints_y"][event_i],
                                      spacepoints["Trec_spacepoints_z"][event_i]], axis=-1))
    
    # true edep spacepoints
    starts = np.stack([spacepoints["TrueEDep_spacepoints_startx"][event_i],
                       spacepoints["TrueEDep_spacepoints_starty"][event_i],
                       spacepoints["TrueEDep_spacepoints_startz"][event_i]], axis=-1)
    midpoints = np.stack([(spacepoints["TrueEDep_spacepoints_startx"][event_i] + spacepoints["TrueEDep_spacepoints_endx"][event_i])/2,
                          (spacepoints["TrueEDep_spacepoints_starty"][event_i] + spacepoints["TrueEDep_spacepoints_endy"][event_i])/2,
                          (spacepoints["TrueEDep_spacepoints_startz"][event_i] + spacepoints["TrueEDep_spacepoints_endz"][event_i])/2], axis=-1)
    ends = np.stack([spacepoints["TrueEDep_spacepoints_endx"][event_i],
                     spacepoints["TrueEDep_spacepoints_endy"][event_i],
                     spacepoints["TrueEDep_spacepoints_endz"][event_i]], axis=-1)
    TrueEDep_spacepoints.append(np.concatenate([starts, midpoints, ends], axis=0))
    # assuming a third of the energy at the start, midpoint, and end
    TrueEDep_spacepoints_edep.append(np.concatenate([spacepoints["TrueEDep_spacepoints_edep"][event_i]/3,
                                                    spacepoints["TrueEDep_spacepoints_edep"][event_i]/3,
                                                    spacepoints["TrueEDep_spacepoints_edep"][event_i]/3], axis=0))
    
    # true gamma 1 and 2 spacepoints
    # see https://github.com/uboone/ubana/blob/3df782aae5dc129c48071317654e9cafd5a32441/ubana/searchingfornues/Selection/AnalysisTools/Pi0TruthAnalysis_tool.cc#L237
    true_gamma1_EDep_spacepoints.append(np.stack([true_pi0_spacepoints["pi0truth_gamma1_xpos"][event_i],
                                                  true_pi0_spacepoints["pi0truth_gamma1_ypos"][event_i],
                                                  true_pi0_spacepoints["pi0truth_gamma1_zpos"][event_i]], axis=-1))
    true_gamma1_EDep_spacepoints_edep.append(true_pi0_spacepoints["pi0truth_gamma1_edep"][event_i])
    true_gamma2_EDep_spacepoints.append(np.stack([true_pi0_spacepoints["pi0truth_gamma2_xpos"][event_i],
                                                  true_pi0_spacepoints["pi0truth_gamma2_ypos"][event_i],
                                                  true_pi0_spacepoints["pi0truth_gamma2_zpos"][event_i]], axis=-1))
    true_gamma2_EDep_spacepoints_edep.append(true_pi0_spacepoints["pi0truth_gamma2_edep"][event_i])
    
#del spacepoints
#del true_pi0_spacepoints
#del f


In [9]:
true_pi0_spacepoints["pi0truth_gamma1_xpos"]

array([218.22162  , 158.6892   , 102.08528  , 155.6318   , 198.95781  ,
        31.07753  , 132.83246  ,  26.367928 ,  54.834698 , 157.30055  ,
        66.08887  ,  68.50515  , 234.80043  ,  96.92556  ,  12.359787 ,
        30.6703   ,  61.618317 ,         inf, 202.33508  ,  37.855045 ,
       223.34445  , 136.06403  ,  49.641663 , 210.74515  ,  21.109722 ,
       181.23203  , 165.60242  , 237.72614  , 253.30698  , 217.83026  ,
        13.053701 , 151.45673  ,  53.839127 ,   5.1448617,  58.56993  ,
       204.61064  , 134.29562  , 142.7147   , 203.11066  , 212.24866  ,
         7.3955717, 201.25056  ,  89.28931  , 163.11842  , 111.062164 ,
       231.07333  ,  68.67175  , 215.46494  ,  86.69516  , 160.78839  ,
        92.334625 , 186.63608  , 105.223015 ,  -1.113327 , 196.84174  ,
        39.823536 ,  99.3475   ,  78.89549  ,         inf, 125.171295 ,
       170.67676  , 124.8833   ,  17.806763 ,  87.94478  ,  90.81964  ,
         7.7598534, 193.32092  , 200.0125   , 240.63171  , 208.5

In [6]:
TrueEDep_spacepoints

[array([[220.3394928 ,   9.12491035, 513.10168457],
        [220.33963013,   9.12218475, 513.1027832 ],
        [220.34103394,   9.09428883, 513.11401367],
        ...,
        [216.24050903, -22.01823997, 515.36669922],
        [216.23884583, -22.01652718, 515.36553955],
        [216.23872375, -22.01630211, 515.36566162]]),
 array([[158.74267578, 110.61690521, 595.96520996],
        [158.74000549, 110.6193161 , 595.96704102],
        [158.73739624, 110.62165833, 595.96887207],
        ...,
        [182.16589355,  34.95254898, 745.1552124 ],
        [182.16589355,  34.95266342, 745.15509033],
        [182.16578674,  34.95306396, 745.1541748 ]]),
 array([[109.68320465,  67.95529175,  12.9399395 ],
        [109.68322754,  67.95527649,  12.93999958],
        [109.69458771,  67.94753265,  12.97044468],
        ...,
        [ 83.88840485,  87.97715759,  34.26058578],
        [ 83.89040375,  87.97766876,  34.2609787 ],
        [ 83.89047241,  87.97828674,  34.26052856]]),
 array([[151.206405

In [7]:
true_gamma2_EDep_spacepoints

[array([224.3013   ,   4.1395807, 509.4018   ], dtype=float32),
 array([158.663   ,  86.768234, 606.26746 ], dtype=float32),
 array([108.95227,  68.48935,  13.59315], dtype=float32),
 array([159.39305 , -82.381256, 914.5199  ], dtype=float32),
 array([190.92513 , -21.284508, 286.39954 ], dtype=float32),
 array([30.770016, 20.840647, 40.73248 ], dtype=float32),
 array([ 120.8945 , -112.35296,  497.7923 ], dtype=float32),
 array([ 46.63161, -16.05633, 559.85376], dtype=float32),
 array([ 62.13595,  75.43652, 878.8169 ], dtype=float32),
 array([190.71323 , -57.707817, 476.96198 ], dtype=float32),
 array([ 65.853905, 115.26872 , 749.0934  ], dtype=float32),
 array([inf, inf, inf], dtype=float32),
 array([242.52103, -76.59889, 800.76306], dtype=float32),
 array([ 80.9323  , 108.47773 ,  23.820553], dtype=float32),
 array([ 23.529655, 104.14157 , 471.40576 ], dtype=float32),
 array([ 13.059188,  49.820225, 107.36759 ], dtype=float32),
 array([ 57.720173,  29.743427, 408.33282 ], dtype=float3

# Downsample Spacepoints

In [8]:
close_to_reco_nu_vtx_threshold = 200
recalculate_downsampling = True

if recalculate_downsampling:
    downsampled_Tcluster_spacepoints = {}
    downsampled_Trec_spacepoints = {}
    downsampled_TrueEDep_spacepoints = {}
    downsampled_true_gamma1_EDep_spacepoints = {}
    downsampled_true_gamma2_EDep_spacepoints = {}
    for event_i in tqdm(range(num_events)):

        nearby_reco_nu_vtx_indices = np.where(np.sqrt((Tcluster_spacepoints[event_i][:, 0] - reco_nu_vtx[event_i][0])**2
                                                    + (Tcluster_spacepoints[event_i][:, 1] - reco_nu_vtx[event_i][1])**2
                                                    + (Tcluster_spacepoints[event_i][:, 2] - reco_nu_vtx[event_i][2])**2) < close_to_reco_nu_vtx_threshold)[0]
        Tcluster_spacepoints_near_reco_nu_vtx = Tcluster_spacepoints[event_i][nearby_reco_nu_vtx_indices, :]
        downsampled_Tcluster_spacepoints[event_i] = fps_clustering_downsample(Tcluster_spacepoints_near_reco_nu_vtx, 500)

        downsampled_Trec_spacepoints[event_i] = fps_clustering_downsample(Trec_spacepoints[event_i], 200)

        downsampled_TrueEDep_spacepoints[event_i] = energy_weighted_density_sampling(TrueEDep_spacepoints[event_i], TrueEDep_spacepoints_edep[event_i], 500)

        downsampled_true_gamma1_EDep_spacepoints[event_i] = energy_weighted_density_sampling(true_gamma1_EDep_spacepoints[event_i], true_gamma1_EDep_spacepoints_edep[event_i], 500)
        downsampled_true_gamma2_EDep_spacepoints[event_i] = energy_weighted_density_sampling(true_gamma2_EDep_spacepoints[event_i], true_gamma2_EDep_spacepoints_edep[event_i], 500)

    with open("downsampled_spacepoints.pkl", "wb") as f:
        pickle.dump((downsampled_Tcluster_spacepoints, downsampled_Trec_spacepoints, downsampled_TrueEDep_spacepoints, downsampled_true_gamma1_EDep_spacepoints, downsampled_true_gamma2_EDep_spacepoints), f)
else:
    with open("downsampled_spacepoints.pkl", "rb") as f:
        downsampled_Tcluster_spacepoints, downsampled_Trec_spacepoints, downsampled_TrueEDep_spacepoints = pickle.load(f)


  0%|          | 0/70 [00:00<?, ?it/s]

points:  [[220.3394928    9.12491035 513.10168457]
 [220.33963013   9.12218475 513.1027832 ]
 [220.34103394   9.09428883 513.11401367]
 ...
 [216.24050903 -22.01823997 515.36669922]
 [216.23884583 -22.01652718 515.36553955]
 [216.23872375 -22.01630211 515.36566162]]
len(points):  78396
points:  [218.22162    -3.6083694 513.3529   ]
len(points):  3


TypeError: object of type 'numpy.float32' has no len()

# Categorize Spacepoints

In [6]:
len(downsampled_Tcluster_spacepoints)

70

In [7]:
real_nu_reco_nu_downsampled_spacepoints = []
real_nu_reco_cosmic_downsampled_spacepoints = []
real_cosmic_reco_nu_downsampled_spacepoints = []
real_cosmic_reco_cosmic_downsampled_spacepoints = []

real_gamma1_downsampled_spacepoints = []
real_gamma2_downsampled_spacepoints = []

close_to_true_nu_spacepoint_threshold = 5
close_to_reco_nu_spacepoint_threshold = 5
close_to_true_gamma_spacepoint_threshold = 5

for event_i in range(num_events):

    if len(downsampled_Tcluster_spacepoints[event_i]) == 0:
        real_nu_reco_cosmic_downsampled_spacepoints.append(np.empty((0, 3)))
        real_nu_reco_nu_downsampled_spacepoints.append(np.empty((0, 3)))
        real_cosmic_reco_nu_downsampled_spacepoints.append(np.empty((0, 3)))
        real_cosmic_reco_cosmic_downsampled_spacepoints.append(np.empty((0, 3)))
        real_gamma1_downsampled_spacepoints.append(np.empty((0, 3)))
        real_gamma2_downsampled_spacepoints.append(np.empty((0, 3)))
        continue

    # for T_cluster spacepoints, noting distances to true nu and reco nu spacepoints, and which are close to the reco nu vtx
    min_truth_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_TrueEDep_spacepoints[event_i][:, :3])
    min_reco_nu_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_Trec_spacepoints[event_i][:, :3])
    min_true_gamma1_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_true_gamma1_EDep_spacepoints[event_i][:, :3])
    min_true_gamma2_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_true_gamma2_EDep_spacepoints[event_i][:, :3])

    # assign features to spacepoints here
    close_to_truth_indices = np.where(min_truth_dists < close_to_true_nu_spacepoint_threshold)[0]
    far_from_truth_indices = np.where(min_truth_dists >= close_to_true_nu_spacepoint_threshold)[0]
    close_to_reco_nu_indices = np.where(min_reco_nu_dists < close_to_reco_nu_spacepoint_threshold)[0]
    far_from_reco_nu_indices = np.where(min_reco_nu_dists >= close_to_reco_nu_spacepoint_threshold)[0]
    close_to_true_gamma1_indices = np.where(min_true_gamma1_dists < close_to_true_gamma_spacepoint_threshold)[0]
    close_to_true_gamma2_indices = np.where(min_true_gamma2_dists < close_to_true_gamma_spacepoint_threshold)[0]

    # categorize spacepoints here
    real_nu_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, close_to_truth_indices)
    real_nu_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, close_to_truth_indices)
    real_cosmic_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, far_from_truth_indices)
    real_cosmic_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, far_from_truth_indices)
    real_gamma1_indices = close_to_true_gamma1_indices
    real_gamma2_indices = close_to_true_gamma2_indices

    real_nu_reco_cosmic_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_nu_reco_cosmic_indices, :])
    real_nu_reco_nu_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_nu_reco_nu_indices, :])
    real_cosmic_reco_nu_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_cosmic_reco_nu_indices, :])
    real_cosmic_reco_cosmic_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_cosmic_reco_cosmic_indices, :])
    real_gamma1_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_gamma1_indices, :])
    real_gamma2_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_gamma2_indices, :])


# Plot in 3D

In [ ]:
include_non_downsampled_points = True

index = 61

run = rse["run"][index]
subrun = rse["subrun"][index]
event = rse["event"][index]

print("run, subrun, event:", run, subrun, event)


fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])

# these are only added to set the camera at a better position
fig.add_trace(go.Scatter3d(
    x=expanded_detector_boundary_points[:, 2],
    y=expanded_detector_boundary_points[:, 0],
    z=expanded_detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=0.2,
        color='black',
        opacity=0.8
    ),
    name='Expanded TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=detector_boundary_points[:, 2],
    y=detector_boundary_points[:, 0],
    z=detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='black',
        opacity=0.8
    ),
    name='TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=[reco_nu_vtx[index][2]],
    y=[reco_nu_vtx[index][0]],
    z=[reco_nu_vtx[index][1]],
    mode='markers',
    marker=dict(size=10, color='purple', opacity=1),
    name='Reco Neutrino Vertex',
    visible='legendonly'
))

fig.add_trace(go.Scatter3d(
    x=[true_nu_vtx[index][2]],
    y=[true_nu_vtx[index][0]],
    z=[true_nu_vtx[index][1]],
    mode='markers',
    marker=dict(size=10, color='green', opacity=1),
    name='True Neutrino Vertex',
    visible='legendonly'

))


if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=Tcluster_spacepoints[index][:, 2],
        y=Tcluster_spacepoints[index][:, 0],
        z=Tcluster_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color="blue",
            opacity=0.8
        ),
        name='Tcluster Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_Tcluster_spacepoints[index][:, 2],
    y=downsampled_Tcluster_spacepoints[index][:, 0],
    z=downsampled_Tcluster_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color="blue",
        opacity=0.8
    ),
    name='Downsampled Tcluster Spacepoints',
    visible='legendonly'
))

if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=Trec_spacepoints[index][:, 2],
        y=Trec_spacepoints[index][:, 0],
        z=Trec_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color='red',
            opacity=0.8
        ),
        name='Trec Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_Trec_spacepoints[index][:, 2],
    y=downsampled_Trec_spacepoints[index][:, 0],
    z=downsampled_Trec_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='red',
        opacity=0.8
    ),
    name='Downsampled Trec Spacepoints',
    visible='legendonly'
))

if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=TrueEDep_spacepoints[index][:, 2],
        y=TrueEDep_spacepoints[index][:, 0],
        z=TrueEDep_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color='orange',
            opacity=0.8
        ),
        name='TrueEDep Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_TrueEDep_spacepoints[index][:, 2],
    y=downsampled_TrueEDep_spacepoints[index][:, 0],
    z=downsampled_TrueEDep_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='orange',
        opacity=0.8
    ),
    name='Downsampled TrueEDep Spacepoints',
    visible='legendonly'
))

fig.add_trace(go.Scatter3d(
    x=real_nu_reco_nu_downsampled_spacepoints[index][:, 2],
    y=real_nu_reco_nu_downsampled_spacepoints[index][:, 0],
    z=real_nu_reco_nu_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='orange',
        opacity=0.8
    ),
    name='Real Nu Reco Nu Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 2],
    y=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 0],
    z=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='red',
        opacity=0.8
    ),
    name='Real Nu Reco Cosmic Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 2],
    y=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 0],
    z=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='brown',
        opacity=0.8
    ),
    name='Real Cosmic Reco Nu Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 2],
    y=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 0],
    z=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
        opacity=0.8
    ),
    name='Real Cosmic Reco Cosmic Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_gamma1_downsampled_spacepoints[index][:, 2],
    y=real_gamma1_downsampled_spacepoints[index][:, 0],
    z=real_gamma1_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='lightgreen',
        opacity=0.8
    ),
    name='Real Gamma 1 Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_gamma2_downsampled_spacepoints[index][:, 2],
    y=real_gamma2_downsampled_spacepoints[index][:, 0],
    z=real_gamma2_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='green',
        opacity=0.8
    ),
    name='Real Gamma 2 Spacepoints',
))

fig.update_layout(
    scene=dict(
        xaxis_title='z',
        yaxis_title='x',
        zaxis_title='y',
        aspectratio=dict(
            x=5,
            y=3,
            z=1
        ),
    ),
    width=2000,
    height=1200,
    autosize=False,
    scene_camera=dict(
        eye=dict(x=-1.5, y=-1.5, z=1.5)
    )
)

fig.show(renderer="browser")


run, subrun, event: 20082 19 990
